In [1]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from datasets import load_dataset

In [5]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

In [72]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [50]:
map_dict = {0:"ira", 1:"miedo", 2:"tristeza", 3:"alegria", 4:"asco", 5:"sorpresa", 6:"otra"}
map_dict_inverse = {v: k for k, v in map_dict.items()}

In [ ]:
emoevent_df = pd.read_csv('../Data/Emoevent/emoevent.csv')
emoevent_df = emoevent_df.rename(columns={"tweet": "text"})
emoevent_df['label'] = emoevent_df["emocion"].map(map_dict_inverse)
emoevent_df = emoevent_df[['text', 'label']]
emoevent_df.to_csv('../Data/Emoevent/emoevent_hugging.csv', index=False)

In [2]:
dataset = load_dataset("csv", data_files='../Data/Emoevent/emoevent_hugging.csv', split = 'train')

Using custom data configuration default-cbf19c6efdd1f5b5
Found cached dataset csv (/home/juanjo/.cache/huggingface/datasets/csv/default-cbf19c6efdd1f5b5/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [3]:
splited_dataset = dataset.train_test_split(test_size=0.2)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [8]:
tokenized_data = splited_dataset.map(tokenize_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=7)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [10]:
metric = evaluate.load("accuracy")

In [76]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [77]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    compute_metrics=compute_metrics,
)

In [78]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/juanjo/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6554
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1230
